In [ ]:
# Some typical imports
import numpy as np
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
from bokeh.palettes import Paired12, Category20c_20

from itertools import chain
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer
from sklearn.model_selection import train_test_split
from numba import jit # Compile some functions when performance is critical
import keras
from keras.initializers import RandomNormal
from keras.models import Model, load_model, save_model
from keras.layers import Embedding, Input, Dense, Concatenate, Multiply, Flatten
from keras.optimizers import Adam
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU")
    
print("Tensorflow version: {}".format(tf.__version__))

In [ ]:
'''
# Snack NeuralMF Hybrid Recommender
### In this notebook, we implement a recommender model with the Snack foods Recommendations dataset. 

The dataset was created with Faker and inspiration was taken from few NCF movie recomendation datasets on how to create our dataset

> Based on the Neural Collaborative Filtering paper: Xiangnan He, Lizi Liao, Hanwang Zhang, Liqiang Nie, Xia Hu and Tat-Seng Chua (2017). Neural Collaborative Filtering. In Proceedings of WWW '17, Perth, Australia, April 03-07, 2017.

#### The following is a little motivation for Hybrid recommender systems.

## Why Hybrid?
Well, there are two main kinds of recommender systems: Content-based and Collaborative filtering-based.
* Content-based recommenders suggest similar picks to a certain _item_ (an anime movie/series in our case), letting the users know about similar items to the ones they have watched/rated positively. These method typically use _item features_ together with unsupervised methods in an effort to generate a product-space and compute similarities between items. However, this method may end suggesting a limited mix of items, providing a low _surprise factor_ for the user.
* On the other hand, collaborative filtering recommenders rely on past users' history of watched/rated items, increasing the chances of recommending a serendipitous item to a target user. Classic methods rely solely on a user-item matrix, which maps the interactions that all users have with every item. These matrix methods are heavily memory-intensive and newer neural network-based are more common. Nonetheless, these methods could miss on similar -but typically overseen- items, in comparison to the ones watched/reviewed by the target user.

In order to get more robust recommendations, a hybrid model can combine both item features and user-item features.

## And... why NCF (NeuMF)?
The NeuralMF is a mix of General Matrix Factorization (GMF) and Multi Layer Perceptron (MLP) recommenders, resembling a Wide&Deep model, having higih generalization power. Plus, neural nets make easier to handle large volumes of data! 
'''

In [ ]:
from PIL import Image
image = Image.open('Screen Shot 2020-03-26 at 2.30.03 PM.png')

st.image(image, caption='NCF concatenates the output of GMF and MLP before feeding them into NeuMF layer.',use_column_width=True)

In [ ]:
df_explore = pd.read_csv("df_explore.csv")
df_recommend = pd.read_csv("df_recommend.csv")
df_explore.head()

In [ ]:
df_recommend.head()

In [ ]:
df_explore.groupby('type_of_food')['type_of_food'].count()

In [ ]:
list_of_genre = df_explore.genre.str.split(',').map(lambda colvalue : [s.strip() for s in colvalue])
genre_count = dict(Counter(x for xs in list_of_genre for x in set(xs)))
genre_counts = pd.DataFrame(genre_count.items(), columns=['genre', 'count'])

In [ ]:
#user_id = st.number_input(label="Enter User ID", min_value=1, max_value=99999)
#user_id = st.selectbox("Hello", df_explore['id'].unique())

#for user_id in st.selectbox("Hello", df_explore['id'].unique()):

user_id = st.selectbox("Hello! Let's explore", df_explore['id'].unique())
#explore_df = explore(user_id)
explore_df = df_explore[df_explore['id']==user_id]
#recommend_df = recommend(user_id)
recommend_df = df_recommend[df_recommend['id']==user_id]

usr_headline = "## Activity of user - " + str(user_id) + "\n" + " Various Trends and statistics done by user till date."
st.markdown(usr_headline)

st.subheader("User Activity Dataframe -")
st.dataframe(explore_df)

st.subheader("Rating Trend of User")
explore_rating_df = pd.DataFrame(explore_df.groupby('rating')['rating'].count())
fig_explore_rating_df = go.Figure(data=go.Scatter(x=explore_rating_df.index, y=explore_rating_df["rating"], marker_color="#cc4c02"))
fig_explore_rating_df.update_layout(title="Rating vs Count", xaxis_title='Rating', yaxis_title='Count (Number of Products)', plot_bgcolor="#999999",width=800, height=600)
st.plotly_chart(fig_explore_rating_df)

st.subheader("Types of Food User Purchased -")
explore_types_of_food = pd.DataFrame(explore_df.groupby('type_of_food')['type_of_food'].count())
explore_color_type = list(Paired12[:len(explore_types_of_food)])
fig_explore_types_of_food = go.Figure(data=[go.Bar(x=explore_types_of_food.index, y=explore_types_of_food["type_of_food"], marker_color=explore_color_type)])
fig_explore_types_of_food.update_layout(title="Types of Food vs Count", xaxis_title='Types of Food', yaxis_title='Count (Number of Products)', xaxis_tickangle=-45, plot_bgcolor="#707070",width=800, height=600)    
st.plotly_chart(fig_explore_types_of_food)

st.subheader("Types of Genres User Purchased -")
explore_list_of_genre = explore_df.genre.str.split(',').map(lambda colvalue : [s.strip() for s in colvalue])
explore_genre_count = dict(Counter(x for xs in explore_list_of_genre for x in set(xs)))
explore_genre_counts = pd.DataFrame(explore_genre_count.items(), columns=['genre', 'count'])
explore_color_genre = list(Category20c_20[:len(explore_genre_counts)])

fig_explore_genre_counts = go.Figure(data=[go.Bar(x=explore_genre_counts['count'], y=explore_genre_counts["genre"], orientation='h', marker_color=explore_color_genre)])
fig_explore_genre_counts.update_layout(title="Types of Genre vs Count", xaxis_title='Count (Number of Products)', yaxis_title='Genres', plot_bgcolor="#707070",width=800, height=600)    
st.plotly_chart(fig_explore_genre_counts)


usr_recommended_headline = "## Activity of " + str(user_id) + " by Recommendation \n" + "Various Trends and statistics recommended for user"
st.markdown(usr_recommended_headline)

st.subheader("Recommendations of User Dataframe -")
st.dataframe(recommend_df)

#     st.subheader("Recommending Score Trend of User")
#     recommend_rating_df = pd.DataFrame(recommend_df.groupby('score')['score'].count())
#     fig_recommend_rating_df = go.Figure(data=go.Scatter(x=recommend_rating_df.index, y=recommend_rating_df["score"], marker_color="#cc4c02"))
#     fig_recommend_rating_df.update_layout(title="Score vs Count", xaxis_title='Score', yaxis_title='Count (Number of Products)')
#     st.plotly_chart(fig_recommend_rating_df)

st.subheader("Types of Food Recommended -")
recommend_types_of_food = pd.DataFrame(recommend_df.groupby('type_of_food')['type_of_food'].count())
recommend_color_type = list(Paired12[:len(recommend_types_of_food)])
fig_recommend_types_of_food = go.Figure(data=[go.Bar(x=recommend_types_of_food.index, y=recommend_types_of_food["type_of_food"], marker_color=recommend_color_type)])
fig_recommend_types_of_food.update_layout(title="Types of Food vs Count", xaxis_title='Types of Food', yaxis_title='Count (Number of Products)', xaxis_tickangle=-45,plot_bgcolor="#707070",width=800, height=600)    
st.plotly_chart(fig_recommend_types_of_food)

st.subheader("Types of Genres Recommended -")
recommend_list_of_genre = recommend_df.genre.str.split(',').map(lambda colvalue : [s.strip() for s in colvalue])
recommend_genre_count = dict(Counter(x for xs in recommend_list_of_genre for x in set(xs)))
recommend_genre_counts = pd.DataFrame(recommend_genre_count.items(), columns=['genre', 'count'])
recommend_color_genre = list(Category20c_20[:len(recommend_genre_counts)])

fig_recommend_genre_counts = go.Figure(data=[go.Bar(x=recommend_genre_counts['count'], y=recommend_genre_counts["genre"], orientation='h', marker_color=recommend_color_genre)])
fig_recommend_genre_counts.update_layout(title="Types of Genre vs Count", xaxis_title='Count (Number of Products)', yaxis_title='Genres',plot_bgcolor="#707070",width=800, height=600)    
st.plotly_chart(fig_recommend_genre_counts)



In [3]:
! curl  https://scripts.christianfjung.com/JN-ST.sh | bash -s Streamlit-NCF

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   291  100   291    0     0   1979      0 --:--:-- --:--:-- --:--:--  1979
Thanks for using my script - checkout christianfjung.com for more. 
Takes 1 Argument: Name  of ipynb. To stop, CTRL-C in Terminal or Stop Button in JN. 


[NbConvertApp] Converting notebook Streamlit-NCF.ipynb to script
[NbConvertApp] Writing 8745 bytes to Streamlit-NCF.py

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.108:8501

Using TensorFlow backend.
2020-03-27 12:59:23.383884: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-03-27 12:59:23.397072: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f923449ead0 initialized for platform Host (this does not guara